In [1]:
import pandas as pd
from grid_search_utils import get_grid_search_results
import itertools

In [2]:
# give directory name
directory = "vague_DS_grid_search"
get_grid_search_results(directory)

     attributes  values  game_size  vocab_size_factor  n_epochs  batch_size  \
0           3.0     4.0       10.0                3.0      60.0        32.0   
1           3.0     4.0       10.0                3.0      60.0        32.0   
2           3.0     4.0       10.0                3.0      60.0        32.0   
3           3.0    16.0       10.0                3.0      60.0        64.0   
4           3.0    16.0       10.0                3.0      60.0        64.0   
..          ...     ...        ...                ...       ...         ...   
211         3.0    16.0       10.0                3.0      60.0        32.0   
212         3.0    16.0       10.0                3.0      60.0        64.0   
213         3.0    16.0       10.0                3.0      60.0        64.0   
214         3.0    16.0       10.0                3.0      60.0        64.0   
215         3.0    16.0       10.0                3.0      60.0        64.0   

     learning_rate  hidden_size  temp_update  tempe

In [3]:
data = pd.read_csv("results_" + directory + ".csv")

## New grid search results

In [4]:
# parameters that were tested
for col in data.columns[:-4]:
    print(col, data[col].unique())    

attributes [3 5]
values [ 4 16]
game_size [10]
vocab_size_factor [3.]
n_epochs [60.]
batch_size [ 32  64 128]
learning_rate [0.0005 0.001 ]
hidden_size [128 256]
temp_update [0.97 0.99]
temperature [1.  1.5 2. ]


In [5]:
data = data[((data['attributes'] == 3) & (data['values'] == 4))
            | ((data['attributes'] == 3) & (data['values'] == 16))
            | ((data['attributes'] == 5) & (data['values'] == 4))]

In [6]:
attributes = [3, 5]
values = [4, 16]
datasets = list(itertools.product(attributes, values))[:3]
index = data.index
for (attributes, values) in datasets:
    print("dataset:", attributes, values)
    per_dataset = data[(data["attributes"] == attributes) & (data["values"] == values)]
    max_test_acc = per_dataset['test_accuracy'].idxmax()
    print(per_dataset.loc[index[max_test_acc]])

dataset: 3 4
attributes             3.000000
values                 4.000000
game_size             10.000000
vocab_size_factor      3.000000
n_epochs              60.000000
batch_size            32.000000
learning_rate          0.001000
hidden_size          256.000000
temp_update            0.970000
temperature            2.000000
train_loss             0.101827
train_accuracy         0.964500
test_loss              0.127935
test_accuracy          0.959722
Name: 22, dtype: float64
dataset: 3 16
attributes             3.000000
values                16.000000
game_size             10.000000
vocab_size_factor      3.000000
n_epochs              60.000000
batch_size            64.000000
learning_rate          0.001000
hidden_size          256.000000
temp_update            0.990000
temperature            2.000000
train_loss             0.095122
train_accuracy         0.973151
test_loss              0.101393
test_accuracy          0.971871
Name: 119, dtype: float64
dataset: 5 4
attributes   

### Batch size
Highest mean accuracies: D(3,4):batch size 32, D(5,4):batch size 64, D(3,16): batch size 128(although insignifantly different than others).

Highest max train accuracies with batch size 64. 

Highest max test accuracies: D(3,4): batch size 32, D(5,4):batch size 64, D(3,16): batch size 64. 

In [9]:
bs_sumstat = data.groupby(['attributes', 'values', 'batch_size'])[['train_accuracy', 'test_accuracy']].describe()
bs_train = bs_sumstat.loc[:,[('train_accuracy', 'max'), ('train_accuracy', 'mean'), ('train_accuracy', 'std')]]
bs_test = bs_sumstat.loc[:,[('test_accuracy', 'max'), ('test_accuracy', 'mean'), ('test_accuracy', 'std')]]
bs_merged = bs_train.merge(bs_test, left_index=True, right_index=True)
bs_merged

train_accuracy                     test_accuracy  \
                                        max      mean       std           max   
attributes values batch_size                                                    
3          4      32               0.964500  0.898187  0.085911      0.959722   
                  64               0.964584  0.863483  0.132902      0.945573   
                  128              0.934750  0.862281  0.108833      0.932422   
           16     32               0.963918  0.938229  0.015965      0.962240   
                  64               0.973151  0.935613  0.017706      0.971871   
                  128              0.958189  0.939355  0.013626      0.958511   
5          4      32               0.973386  0.864359  0.132628      0.971135   
                  64               0.982673  0.906003  0.086190      0.982143   
                  128              0.979927  0.878916  0.143342      0.978155   

                                                  
                                  mean       std  
attributes values batch_size                      
3          4      32          0.890327  0.088165  
                  64          0.850145  0.133712  
                  128         0.852132  0.106479  
           16     32          0.936968  0.015603  
                  64          0.935398  0.017539  
                  128         0.939110  0.013563  
5          4      32          0.863077  0.133838  
                  64          0.904450  0.086863  
                  128         0.876940  0.143670

### Learning rate
Highest maximum: D(3,4) higher train acc with lr 0.0005 (very small difference) but higher test acc w/ 0.001. D(4,5) with lr 0.0005, D(3,16): with lr 0.001

Higher mean accuracies with learning rate 0.0005.

In [11]:
lr_sumstat = data.groupby(['attributes', 'values', 'learning_rate'])[['train_accuracy', 'test_accuracy']].describe()
lr_train = lr_sumstat.loc[:,[('train_accuracy', 'max'), ('train_accuracy', 'mean'), ('train_accuracy', 'std')]]
lr_test = lr_sumstat.loc[:,[('test_accuracy', 'max'), ('test_accuracy', 'mean'), ('test_accuracy', 'std')]]
lr_merged = lr_train.merge(lr_test, left_index=True, right_index=True)
lr_merged

train_accuracy                      \
                                           max      mean       std   
attributes values learning_rate                                      
3          4      0.0005              0.964584  0.905578  0.028376   
                  0.0010              0.964500  0.843723  0.148453   
           16     0.0005              0.963556  0.938680  0.017542   
                  0.0010              0.973151  0.936785  0.013821   
5          4      0.0005              0.982673  0.926640  0.031097   
                  0.0010              0.950365  0.839545  0.160274   

                                test_accuracy                      
                                          max      mean       std  
attributes values learning_rate                                    
3          4      0.0005             0.945573  0.893694  0.025951  
                  0.0010             0.959722  0.834709  0.150160  
           16     0.0005             0.962240  0.938605  0.017428  
                  0.0010             0.971871  0.935712  0.013407  
5          4      0.0005             0.982143  0.925850  0.030535  
                  0.0010             0.948107  0.837128  0.161000

### Hidden size
Highest mean accuracies:                                Highest max accuracies: 

D(3,4) with hidden size 128.                            D(3,4) with hidden size 256.

D(3,16) with hidden size 256.                           D(3,16) with hidden size 256.

D(5,4) with hidden size 128.                            D(5,4) with hidden size 256.

In [12]:
hs_sumstat = data.groupby(['attributes', 'values', 'hidden_size'])[['train_accuracy', 'test_accuracy']].describe()
hs_train = hs_sumstat.loc[:,[('train_accuracy', 'max'), ('train_accuracy', 'mean'), ('train_accuracy', 'std')]]
hs_test = hs_sumstat.loc[:,[('test_accuracy', 'max'), ('test_accuracy', 'mean'), ('test_accuracy', 'std')]]
hs_merged = hs_train.merge(hs_test, left_index=True, right_index=True)
hs_merged

train_accuracy                      \
                                         max      mean       std   
attributes values hidden_size                                      
3          4      128               0.939493  0.894357  0.022803   
                  256               0.964584  0.854944  0.153272   
           16     128               0.960701  0.934364  0.012959   
                  256               0.973151  0.941100  0.017587   
5          4      128               0.969235  0.913595  0.021441   
                  256               0.982673  0.852590  0.167876   

                              test_accuracy                      
                                        max      mean       std  
attributes values hidden_size                                    
3          4      128              0.930729  0.885775  0.020888  
                  256              0.959722  0.842628  0.153679  
           16     128              0.958015  0.933553  0.012390  
                  256              0.971871  0.940765  0.017536  
5          4      128              0.955777  0.912063  0.021159  
                  256              0.982143  0.850915  0.168911

### Temperature
Highest mean and max accuracies with temperature 2. 

In [13]:
t_sumstat = data.groupby(['attributes', 'values', 'temperature'])[['train_accuracy', 'test_accuracy']].describe()
t_train = t_sumstat.loc[:,[('train_accuracy', 'max'), ('train_accuracy', 'mean'), ('train_accuracy', 'std')]]
t_test = t_sumstat.loc[:,[('test_accuracy', 'max'), ('test_accuracy', 'mean'), ('test_accuracy', 'std')]]
t_merged = t_train.merge(t_test, left_index=True, right_index=True)
t_merged

train_accuracy                      \
                                         max      mean       std   
attributes values temperature                                      
3          4      1.0               0.952497  0.846398  0.130104   
                  1.5               0.962229  0.860939  0.130323   
                  2.0               0.964584  0.916615  0.031278   
           16     1.0               0.946880  0.927657  0.012184   
                  1.5               0.963556  0.937763  0.016186   
                  2.0               0.973151  0.947777  0.011837   
5          4      1.0               0.951620  0.842676  0.150314   
                  1.5               0.977138  0.877592  0.138349   
                  2.0               0.982673  0.929010  0.028766   

                              test_accuracy                      
                                        max      mean       std  
attributes values temperature                                    
3          4      1.0              0.943924  0.837131  0.132007  
                  1.5              0.951129  0.850252  0.130199  
                  2.0              0.959722  0.905222  0.030965  
           16     1.0              0.948100  0.927420  0.012617  
                  1.5              0.962240  0.937235  0.016175  
                  2.0              0.971871  0.946822  0.011183  
5          4      1.0              0.952612  0.841554  0.151730  
                  1.5              0.978155  0.875726  0.138886  
                  2.0              0.982143  0.927188  0.028895

### Temperature update
Highest mean and max accuracies for temperature update 0.99.

In [14]:
tu_sumstat = data.groupby(['attributes', 'values', 'temp_update'])[['train_accuracy', 'test_accuracy']].describe()
tu_train = tu_sumstat.loc[:,[('train_accuracy', 'max'), ('train_accuracy', 'mean'), ('train_accuracy', 'std')]]
tu_test = tu_sumstat.loc[:,[('test_accuracy', 'max'), ('test_accuracy', 'mean'), ('test_accuracy', 'std')]]
tu_merged = tu_train.merge(tu_test, left_index=True, right_index=True)
tu_merged

train_accuracy                      \
                                         max      mean       std   
attributes values temp_update                                      
3          4      0.97              0.964500  0.868808  0.108391   
                  0.99              0.964584  0.880493  0.113983   
           16     0.97              0.958144  0.931862  0.013995   
                  0.99              0.973151  0.943603  0.015292   
5          4      0.97              0.966802  0.879349  0.111932   
                  0.99              0.982673  0.886836  0.134158   

                              test_accuracy                      
                                        max      mean       std  
attributes values temp_update                                    
3          4      0.97             0.959722  0.858012  0.108549  
                  0.99             0.951129  0.870391  0.114670  
           16     0.97             0.958015  0.931963  0.014183  
                  0.99             0.971871  0.942355  0.015201  
5          4      0.97             0.964173  0.877444  0.112470  
                  0.99             0.982143  0.885534  0.134975